In [1]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split

# Load semua data

- load data yang berformat npz dan langsung ambil `value` dari key `data`

In [ ]:
images = np.load("numpy_files/images.npz")["data"]
image_names = np.load("numpy_files/image_name.npz", allow_pickle=True)["data"]
gray_feature = np.load("numpy_files/gray_feature.npz")["data"]
canny_feature = np.load("numpy_files/canny_feature.npz")["data"]
local_binary_pattern_feature = np.load("numpy_files/local_binary_pattern_feature.npz")["data"]
cardboard_class = np.load("numpy_files/cardboard_class.npz")["data"]
bbox = np.load("numpy_files/bbox.npz")["data"]
segmentation = np.load("numpy_files/segmentation.npz")["data"]
segmentation_category = np.load("numpy_files/segmentation_category.npz")["data"]

- Cek shape dari setiap data

In [ ]:
print(
    f"""
        images:                 {images.shape}
        image_names:            {image_names.shape}
        gray_feature:           {gray_feature.shape}
        canny_feature:          {canny_feature.shape}
        lbp_feature:            {local_binary_pattern_feature.shape}
        cardboard_class:        {cardboard_class.shape}
        bbox:                   {bbox.shape}
        segmentation:           {segmentation.shape}
        segmentation_category:  {segmentation_category.shape}
      """
)


        images:                 (369, 1, 300, 400, 3)
        images_with_bg:         (369, 1, 300, 400, 3)
        image_names:            (369,)
        gray_feature:           (369, 1, 300, 400)
        canny_feature:          (369, 1, 300, 400)
        lbp_feature:            (369, 1, 300, 400)
        cardboard_class:        (369, 1, 2)
        bbox:                   (369, 5, 4)
        segmentation:           (369, 5, 300, 400)
        segmentation_category:  (369, 5, 3)
      


- Cek total data

In [4]:
total_data = cardboard_class.shape[0]
total_data

369

# Split Dataset

In [5]:
# generate list index [0, 1, 2, ...., total_data-1]
indices = np.arange(total_data)
indices.shape

(369,)

- proses dibawah ini akan menghasilkan index yang akan di split
  - misal raw_index [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  - setelah di split akan menjadi
    - train = [2, 4, 5, 7, 8]
    - test = [1, 3, 10]
    - validation = [6, 9]
  - setelah mendapatkan index untuk setiap subset, data citra dll akan di split berdasarkan index itu tadi

In [6]:
# split dengan perbandingan 80:20 untuk data train dan (test, validation)
(train_indices, temp_indices, train_label, temp_label) = train_test_split(
    indices,
    cardboard_class.argmax(axis=2),
    test_size=0.2,
    random_state=42,
    stratify=cardboard_class.argmax(axis=2),
)

# split 20% tadi menjadi 2 (50:50) untuk data test dan validation
(val_indices, test_indices, val_label, test_label) = train_test_split(
    temp_indices, temp_label, test_size=0.5, random_state=42, stratify=temp_label
)

- buat fungsi untuk menerapkan hasil split index ke data aslinya untuk setiap subset nya

In [7]:
def train_test_val_split(data, train_indices, test_indices, val_indices):
    train_data = data[train_indices]
    test_data = data[test_indices]
    val_data = data[val_indices]

    return train_data, test_data, val_data

- split semua data yang diawal tadi dengan hasil split index `train`, `test`, `val`

In [ ]:
train_images, test_images, val_images = train_test_val_split(
    images, train_indices, test_indices, val_indices
)

train_image_names, test_image_names, val_image_names = train_test_val_split(
    image_names, train_indices, test_indices, val_indices
)

train_gray_feature, test_gray_feature, val_gray_feature = train_test_val_split(
    gray_feature, train_indices, test_indices, val_indices
)

train_canny_feature, test_canny_feature, val_canny_feature = train_test_val_split(
    canny_feature, train_indices, test_indices, val_indices
)

(
    train_local_binary_pattern_feature,
    test_local_binary_pattern_feature,
    val_local_binary_pattern_feature,
) = train_test_val_split(
    local_binary_pattern_feature, train_indices, test_indices, val_indices
)

train_cardboard_class, test_cardboard_class, val_cardboard_class = train_test_val_split(
    cardboard_class, train_indices, test_indices, val_indices
)

train_bbox, test_bbox, val_bbox = train_test_val_split(
    bbox, train_indices, test_indices, val_indices
)

train_segmentation, test_segmentation, val_segmentation = train_test_val_split(
    segmentation, train_indices, test_indices, val_indices
)

train_segmentation_category, test_segmentation_category, val_segmentation_category = (
    train_test_val_split(
        segmentation_category, train_indices, test_indices, val_indices
    )
)

- tampilkan hasil split dataset

In [9]:
print("train_data :", train_images.shape[0], "data")
print("test_data :", test_images.shape[0], "data")
print("val_data :", val_images.shape[0], "data")

train_data : 295 data
test_data : 37 data
val_data : 37 data


- tampilkan distribusi setelah proses split dataset

In [10]:
# ambil banyak data setiap kelas di setiap subset
train_normal = len(train_label[train_label == [0]])
train_defect = len(train_label[train_label == [1]])
test_normal = len(test_label[test_label == [0]])
test_defect = len(test_label[test_label == [1]])
val_normal = len(val_label[val_label == [0]])
val_defect = len(val_label[val_label == [1]])

print("Distribusi Split Data")

print(
    f"""Train Data:

    Normal : {train_normal} data
    Defect : {train_defect} data

    """
)
print(
    f"""Test Data:

    Normal : {test_normal} data
    Defect : {test_defect} data

    """
)
print(
    f"""Validation Data:

    Normal : {val_normal} data
    Defect : {val_defect} data

    """
)

Distribusi Split Data
Train Data:

    Normal : 144 data
    Defect : 151 data

    
Test Data:

    Normal : 18 data
    Defect : 19 data

    
Validation Data:

    Normal : 18 data
    Defect : 19 data

    


# Simpan hasil split dataset

### fungsi untuk mengekspor citra setelah proses split dataset

In [11]:
!rm -rf images-split-data

def export_split_data_to_image(
    image,
    image_name,
    label,
    feature,
    train_indices,
    test_indices,
    val_indices,
    export_path="images-split-data",
):
    # ubah label kelas ke bentuk flatten
    label = label[:,0].argmax(axis=1)
    
    # convert label kelas ke string (bentuk awalnya adalah one hot encoder)
    label = np.where(label == 0, "normal", "defect")

    for subset, indices in zip(["train", "test", "val"],[train_indices, test_indices, val_indices]):
        for i in indices:
            
            # buat direktori untuk menyimpan citra hasil split data
            os.makedirs(os.path.join(export_path, feature, subset, label[i]), exist_ok=True)
            path = os.path.join(export_path, feature, subset, label[i], image_name[i])
            if image[i][0].ndim == 3:
                
                # convert citra menjadi channel BGR, karena opencv saat melakukan imwrite akan mengubah ke RGB lagi
                # jika citra memiliki channel RGB maka saat opencv melakukan imwrite akan disimpan dengan channel BGR
                image[i][0] = cv2.cvtColor(image[i][0], cv2.COLOR_RGB2BGR)
            
            # simpan citra ke path yang telah diberikan
            if np.max(image[i][0]) > 1:
                cv2.imwrite(path, image[i][0])
            else:
                cv2.imwrite(path, (image[i][0]*255).astype('uint8'))

- terapkan fungsi diatas untuk menyimpan citra hasil split setiap fitur

In [ ]:
export_split_data_to_image(
    images,
    image_names,
    cardboard_class,
    "original",
    train_indices,
    test_indices,
    val_indices,
)
export_split_data_to_image(
    gray_feature,
    image_names,
    cardboard_class,
    "gray",
    train_indices,
    test_indices,
    val_indices,
)
export_split_data_to_image(
    canny_feature,
    image_names,
    cardboard_class,
    "canny",
    train_indices,
    test_indices,
    val_indices,
)
export_split_data_to_image(
    local_binary_pattern_feature,
    image_names,
    cardboard_class,
    "lbp",
    train_indices,
    test_indices,
    val_indices,
)

- simpan semua dataset hasil split data ke dalam format npz
- akan memiliki struktur key:value
- memiliki key `train`, `test`, `val` untuk mengakses data setiap subsetnya

In [ ]:
# buat direktori untuk menyimpan hasil split data
os.makedirs("numpy_files/split_data", exist_ok=True)

np.savez_compressed(
    "numpy_files/split_data/split_images.npz",
    train=train_images,
    test=test_images,
    val=val_images,
)
np.savez_compressed(
    "numpy_files/split_data/split_image_name.npz",
    train=train_image_names,
    test=test_image_names,
    val=val_image_names,
)
np.savez_compressed(
    "numpy_files/split_data/split_gray_feature.npz",
    train=train_gray_feature,
    test=test_gray_feature,
    val=val_gray_feature,
)
np.savez_compressed(
    "numpy_files/split_data/split_canny_feature.npz",
    train=train_canny_feature,
    test=test_canny_feature,
    val=val_canny_feature,
)
np.savez_compressed(
    "numpy_files/split_data/split_local_binary_pattern_feature.npz",
    train=train_local_binary_pattern_feature,
    test=test_local_binary_pattern_feature,
    val=val_local_binary_pattern_feature,
)
np.savez_compressed(
    "numpy_files/split_data/split_cardboard_class.npz",
    train=train_cardboard_class,
    test=test_cardboard_class,
    val=val_cardboard_class,
)
np.savez_compressed(
    "numpy_files/split_data/split_bbox.npz",
    train=train_bbox,
    test=test_bbox,
    val=val_bbox,
)
np.savez_compressed(
    "numpy_files/split_data/split_segmentation.npz",
    train=train_segmentation,
    test=test_segmentation,
    val=val_segmentation,
)
np.savez_compressed(
    "numpy_files/split_data/split_segmentation_category.npz",
    train=train_segmentation_category,
    test=test_segmentation_category,
    val=val_segmentation_category,
)